## Entrainement d'un modèle avec la méthode des SVM

In [20]:
import os
import numpy as np
import random
from PIL import Image

### 1) Fonctions de Preprocessing des datasets

In [21]:
AVERAGE_SIZE_IMAGE = (127, 145)  # Thanks to the stats, we know that size of bbox will be (127, 145) -> Average size of labels 

def generate_empty_bbox(image_width, image_height):
    """ 
    Generate an empty box for images without label
    """
    # Generating random coords for the bbox
    x_min = random.randint(0, image_width - AVERAGE_SIZE_IMAGE[0])
    y_min = random.randint(0, image_height - AVERAGE_SIZE_IMAGE[1])
    
    # Compute complete coords of the bbox
    x_max = x_min + AVERAGE_SIZE_IMAGE[0]
    y_max = y_min + AVERAGE_SIZE_IMAGE[1]
    
    return (x_min, y_min, x_max, y_max)

def load_data(image_dir, label_dir):
    """ 
    Create a dict with all the usefull datas of the dataset
    datas = {
        "XXXX" (name of the file) : {
            "img" : image as an array,
            "labels" (data of the labels): {
                "X" index of the label (0,1,...,n) : {
                    "name" : name of the label,
                    "coord" : coord of the label like xmin, ymin, xmax, ymax,
                    "img" : crooped img of the label,
                }
            }
        }
    }
    """
    
    datas = {}

    for image_file in os.listdir(image_dir):
        # Computing name and files paths
        image_path = image_dir + '/' + image_file
        name = image_file.split('.')[0]
        label_path = label_dir + '/' + name + '.csv'
        
        # Import image as array
        image = np.array(Image.open(image_path))

        # Import labels as array 
        with open(label_path, 'r') as file:
            rows = file.readlines()

            label_data = {}
            if rows == ['\n']:  # Create a random empty label to balance model
                # Create random coords for empty label
                xmin, ymin, xmax, ymax = generate_empty_bbox(image.shape[1], image.shape[0])
    
                # Get the cropped image (as array) of the label
                cropped_image = np.array(Image.fromarray(image[ymin:ymax, xmin:xmax]).resize(AVERAGE_SIZE_IMAGE))
               
                label_data[0] = {
                        "name":"empty",
                        "coord": (xmin, ymin, xmax, ymax),
                        "img":cropped_image
                    }
            else:
                for i, row in enumerate(rows):  # One image can contain several labels
                    row = row.strip().split(",")

                    # Compute coords of the label
                    xmin, ymin, xmax, ymax = map(int, row[0:4])

                    # Get the label name
                    class_name = row[4]

                    # Get the cropped image (as array) of the label
                    cropped_image = np.array(Image.fromarray(image[ymin:ymax, xmin:xmax]).resize(AVERAGE_SIZE_IMAGE))
                    
                    # Adding to the json
                    label_data[i] = {
                        "name":class_name,
                        "coord": (xmin, ymin, xmax, ymax),
                        "img":cropped_image
                    }

        datas[name] = {
             "img" : image,
             "labels" : label_data,
        }
       
    return datas

# Dict to convert str class name to int
name_to_int = {
    "danger": 0,
    "interdiction": 1,
    "obligation": 2,
    "stop": 3,
    "ceder": 4,
    "frouge": 5,
    "forange": 6,
    "fvert": 7,
    "ff": 8,
    "empty": 9
}

### 2) Fonction de création des datasets

In [22]:
def create_xy(datas):
    # Creating arrays with all labels datas & classes
    X = []
    Y = []

    for name, data in datas.items():
        for row in data["labels"].values():
            image_as_array = np.array(row["img"]).flatten()
            X.append(image_as_array)
            Y.append(name_to_int[row["name"]])

    X = np.array(X)
    Y = np.array(Y)

    return X, Y

### 3) Création des datasets

In [23]:
# Training dataset
datas_train = load_data("../../data/train/images", "../../data/train/labels")
X_train, Y_train = create_xy(datas=datas_train)

# Validation dataset
datas_val = load_data("../../data/val/images", "../../data/val/labels")
X_val, Y_val = create_xy(datas=datas_val)

### 4) Application de la méthode des SVM

In [24]:
from sklearn import svm

svm_model = svm.SVC(kernel='linear') 
svm_model.fit(X_train, Y_train)
y = svm_model.predict(X_val)

print(f"Taux d'erreur : {np.mean(y != Y_val)}")

Taux d'erreur : 0.226890756302521


### 5) Test de la méthode SVM avec application des caractéristiques HOG

In [27]:
from skimage.feature import hog
from skimage.color import rgb2gray

def extract_hog(datas):
    # Creating X array with all HOG information of images
    X = []

    for name, data in datas.items():
        for row in data["labels"].values():
            image_as_array = np.array(hog(rgb2gray(row["img"]))).flatten()
            X.append(image_as_array)

    return np.array(X)


# Update training dataset
X_train_HOG = extract_hog(datas=datas_train)

# Update validation dataset
X_val_HOG = extract_hog(datas=datas_val)

In [28]:
svm_model = svm.SVC(kernel='linear') 
svm_model.fit(X_train_HOG, Y_train)
y_HOG = svm_model.predict(X_val_HOG)

print(f"Taux d'erreur : {np.mean(y_HOG != Y_val)}")

Taux d'erreur : 0.15966386554621848


### 6) Test de la méthode SVM avec application des LPB

In [34]:
import cv2

def extract_SIFT(datas):
    # Creating X array with all HOG information of images
    X = []
    sift = cv2.SIFT_create()

    for name, data in datas.items():
        for row in data["labels"].values():
            gray_image = cv2.cvtColor(data["img"], cv2.COLOR_RGB2GRAY)
            keypoints, descriptors = sift.detectAndCompute(gray_image, None)
            if descriptors is not None:
                X.append(descriptors.flatten())

    return np.array(X)


# Update training dataset
X_train_LBP = extract_SIFT(datas=datas_train)

# Update validation dataset
X_val_LBP = extract_SIFT(datas=datas_val)

c:\Users\victo\AppData\Local\Programs\Python\Python312\Lib\site-packages\skimage\feature\texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1071,) + inhomogeneous part.

In [ ]:
svm_model = svm.SVC(kernel='linear') 
svm_model.fit(X_train_LBP, Y_train)
y_LBP = svm_model.predict(X_val_LBP)

print(f"Taux d'erreur : {np.mean(y_LBP != Y_val)}")